In [1]:
!pip install google-generativeai python-dotenv

  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached google_api_python_client-2.187.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached protobuf-6.33.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached grpcio_status-1.76.

In [2]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [4]:
genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel("gemini-2.5-flash")

query = "What is the weather like in Hyderabad right now and in 10 days from now?"
resp = model.generate_content(query)

In [5]:
resp.text

"Okay, let's get you the weather for Hyderabad, India!\n\n**Right now (as of my last update - please note live conditions can change quickly):**\n\n*   **Temperature:** Approximately **27-29°C (81-84°F)**\n*   **Conditions:** Generally **hazy sunshine** or **partly cloudy**.\n*   **Humidity:** Moderate to high, around **60-70%**.\n*   **Wind:** Light.\n\n---\n\n**Looking ahead to 10 days from now (around November 5th-6th):**\n\nPlease keep in mind that long-range forecasts are subject to change and become less accurate the further out they go.\n\n*   **Temperature:** The forecast generally indicates a continuation of warm, pleasant weather.\n    *   **Daily Highs:** Expected to be around **29-31°C (84-88°F)**.\n    *   **Nighttime Lows:** Expected to be around **19-21°C (66-70°F)**.\n*   **Conditions:** Likely to be **mostly sunny to partly cloudy**, with a very low chance of rain. The hazy conditions often seen in Hyderabad can persist.\n*   **Summary:** Expect warm days and comfortab

In [7]:
#Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = [
    "AI transforminf world",
    "Machine Learning is part of AI",
    "Cooking is easy and fun",
    "AI will shape the future",
    "Chicken recipes are very popular"
]

user_sentence = "future of AI"

#Create Bag of Words
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(documents + [user_sentence])

bow_array = bow_matrix.toarray()
vocab = vectorizer.get_feature_names_out()

#Print Vocab
print("Vocabulary :", vocab)

#print doc wise bow
print("\n -- Document wise bow representation -- \n")
for idx, doc in enumerate(documents):
    print(f"Document {idx} : {doc}")
    print("Bow Vector :", bow_array[idx])
    

#Last row is user input vector
user_vec = bow_matrix[-1]
docs_vecs = bow_matrix[:-1]

#compute cosine similarity
similarities = cosine_similarity(user_vec, docs_vecs).flatten()

#Get closest doc
closest_index = similarities.argmax()
closest_score = similarities[closest_index]
closest_doc = similarities[closest_index]

print("\nUser Query :", user_sentence)
print("\nClosest Match :", closest_doc)
print("\nSimilarity Score :", round(closest_score,4))

for word, count in zip(vocab, bow_array[-1]) :
    if count > 0 :
        print(f"{word} : {count}")

Vocabulary : ['ai' 'and' 'are' 'chicken' 'cooking' 'easy' 'fun' 'future' 'is'
 'learning' 'machine' 'of' 'part' 'popular' 'recipes' 'shape' 'the'
 'transforminf' 'very' 'will' 'world']

 -- Document wise bow representation -- 

Document 0 : AI transforminf world
Bow Vector : [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
Document 1 : Machine Learning is part of AI
Bow Vector : [1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
Document 2 : Cooking is easy and fun
Bow Vector : [0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Document 3 : AI will shape the future
Bow Vector : [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0]
Document 4 : Chicken recipes are very popular
Bow Vector : [0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0]

User Query : future of AI

Closest Match : 0.5163977794943223

Similarity Score : 0.5164
ai : 1
future : 1
of : 1


In [8]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

documents = [
    "Lions live in the wild and hunt for food",
    "Dogs are common indoor pets and are friends",
    "Cars are used for transportation on roads",
    "Cats are also popular indoor pets",
    "Elephants live in the forest and are vey large animals"
]


# tokenize
tokenized_docs = [simple_preprocess(doc) for doc in documents]

#train
model = Word2Vec(sentences=tokenized_docs, vector_size=50, min_count=1)

#average embedding
def get_avg_embedding(model, doc_tokens) :
    vectors = [model.wv[w] for w in doc_tokens if w in model.wv]
    return np.mean(vectors, axis=0)

# all documents embeddings 
doc_embeddings = np.array([get_avg_embedding(model,doc) for doc in tokenized_docs])

#Input
query_raw = "large"
query_tokens = simple_preprocess(query_raw)

if query_tokens and query_tokens[0] in model.wv :
    query_embedding = model.wv[query_tokens[0]]
else :
    query_embedding = np.zeros(model.vector_size)


#similarity
sims = cosine_similarity([query_embedding], doc_embeddings)[0]
best_idx = np.argmax(sims)

print("Query word :", query_raw)
print("\nDocument Ranking :")
for score, doc in sorted(zip(sims, documents), reverse = True) :
    print(f"(score:.3f) -> {doc}")

print("\nMost relevant document :")
print(documents[best_idx])
    


ModuleNotFoundError: No module named 'gensim'

In [9]:
pip show gensim

Note: you may need to restart the kernel to use updated packages.


In [10]:
import sys
print(sys.executable)

D:\MySoftwares\Anaconda-distri\python.exe
